In [5]:
import os
os.getcwd()
os.chdir("../")

In [7]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
from datetime import datetime
from utils.dbmanager import create_sql_connection, ingest_data_to_sql
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
from scipy import stats
import random
random.seed(42)

In [4]:
# Define correct data types based on SQL Table
data_types = {
    "ID": "int64",
    "Client_Income": "float64",
    "Car_Owned": "int64",
    "Bike_Owned": "int64",
    "Active_Loan": "int64",
    "House_Own": "int64",
    "Child_Count": "int64",
    "Credit_Amount": "float64",
    "Loan_Annuity": "float64",
    "Accompany_Client": "string",
    "Client_Income_Type": "string",
    "Client_Education": "string",
    "Client_Marital_Status": "string",
    "Client_Gender": "string",
    "Loan_Contract_Type": "string",
    "Client_Housing_Type": "string",
    "Population_Region_Relative": "float64",
    "Age_Days": "float64",
    "Employed_Days": "float64",
    "Registration_Days": "float64",
    "ID_Days": "float64",
    "Own_House_Age": "float64",
    "Mobile_Tag": "int64",
    "Homephone_Tag": "int64",
    "Workphone_Working": "int64",
    "Client_Occupation": "string",
    "Client_Family_Members": "int64",
    "Client_City_Rating": "float64",
    "Application_Process_Day": "int64",
    "Application_Process_Hour": "int64",
    "Client_Permanent_Match_Tag": "string",
    "Client_Contact_Work_Tag": "string",
    "Type_Organization": "string",
    "Score_Source_1": "float64",
    "Score_Source_2": "float64",
    "Score_Source_3": "float64",
    "Social_Circle_Default": "float64",
    "Phone_Change": "float64",
    "Credit_Bureau": "float64",
    "Defaulters": "int64"
}

In [5]:
server = 'nbfi-loan-def-prediction.database.windows.net'     
database = 'NBFIDB'  
username = 'adminuser'       
password = 'Rambo@12345' 

# Establish connection
conn = create_sql_connection(server, database, username, password)

Connection established successfully!


### Loading Data

In [6]:
data = pd.read_csv(r'../data/NBFIData.csv')
data.shape

(121787, 40)

In [7]:
data.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters
0,12142509,6750.0,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,Home,0.028663,13957.0,1062.0,6123.0,383.0,NaN,1,1,0,Sales,2.0,2.0,6.0,17.0,Yes,Yes,Self-employed,0.568066,0.478787,NaN,0.0186,63.0,NaN,0
1,12138936,20250.0,1.0,0.0,1.0,NaN,0.0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,Home,0.008575,14162.0,4129.0,7833.0,21.0,0.0,1,0,1,NaN,2.0,2.0,3.0,10.0,Yes,Yes,Government,0.563360,0.215068,NaN,NaN,NaN,NaN,0
2,12181264,18000.0,0.0,0.0,1.0,0.0,1.0,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,Family,0.022800,16790.0,5102.0,NaN,331.0,NaN,1,0,0,Realty agents,2.0,2.0,4.0,NaN,Yes,Yes,Self-employed,NaN,0.552795,0.329655,0.0742,277.0,0.0,0
3,12188929,15750.0,0.0,0.0,1.0,1.0,0.0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,Home,0.010556,23195.0,365243.0,NaN,775.0,NaN,1,0,0,NaN,2.0,3.0,2.0,15.0,Yes,Yes,XNA,NaN,0.135182,0.631355,NaN,1700.0,3.0,0
4,12133385,33750.0,1.0,0.0,1.0,0.0,2.0,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,Home,0.020713,11366.0,2977.0,5516.0,4043.0,6.0,1,0,0,Laborers,4.0,1.0,3.0,NaN,Yes,Yes,Business Entity Type 3,0.508199,0.301182,0.355639,0.2021,674.0,1.0,0


In [8]:
# Convert columns to correct data types
for column, dtype in data_types.items():
    if dtype == "int64":
        data[column] = pd.to_numeric(data[column], errors="coerce").fillna(0).astype("int64")
    elif dtype == "float64":
        data[column] = pd.to_numeric(data[column], errors="coerce").fillna(0.0).astype("float64")
    elif dtype == "string":
        data[column] = data[column].astype("string").fillna("Unknown")

# Verify types after conversion
print(data.dtypes)

ID                                     int64
Client_Income                        float64
Car_Owned                              int64
Bike_Owned                             int64
Active_Loan                            int64
House_Own                              int64
Child_Count                            int64
Credit_Amount                        float64
Loan_Annuity                         float64
Accompany_Client              string[python]
Client_Income_Type            string[python]
Client_Education              string[python]
Client_Marital_Status         string[python]
Client_Gender                 string[python]
Loan_Contract_Type            string[python]
Client_Housing_Type           string[python]
Population_Region_Relative           float64
Age_Days                             float64
Employed_Days                        float64
Registration_Days                    float64
ID_Days                              float64
Own_House_Age                        float64
Mobile_Tag

In [9]:
data.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters
0,12142509,6750.0,0,0,1,0,0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,Home,0.028663,13957.0,1062.0,6123.0,383.0,0.0,1,1,0,Sales,2,2.0,6,17,Yes,Yes,Self-employed,0.568066,0.478787,0.000000,0.0186,63.0,0.0,0
1,12138936,20250.0,1,0,1,0,0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,Home,0.008575,14162.0,4129.0,7833.0,21.0,0.0,1,0,1,Unknown,2,2.0,3,10,Yes,Yes,Government,0.563360,0.215068,0.000000,0.0000,0.0,0.0,0
2,12181264,18000.0,0,0,1,0,1,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,Family,0.022800,16790.0,5102.0,0.0,331.0,0.0,1,0,0,Realty agents,2,2.0,4,0,Yes,Yes,Self-employed,0.000000,0.552795,0.329655,0.0742,277.0,0.0,0
3,12188929,15750.0,0,0,1,1,0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,Home,0.010556,23195.0,365243.0,0.0,775.0,0.0,1,0,0,Unknown,2,3.0,2,15,Yes,Yes,XNA,0.000000,0.135182,0.631355,0.0000,1700.0,3.0,0
4,12133385,33750.0,1,0,1,0,2,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,Home,0.020713,11366.0,2977.0,5516.0,4043.0,6.0,1,0,0,Laborers,4,1.0,3,0,Yes,Yes,Business Entity Type 3,0.508199,0.301182,0.355639,0.2021,674.0,1.0,0


In [10]:
# data.isna().sum()

In [12]:
# Ingest Data into Bronze Table
if conn:
    ingest_data_to_sql(conn, data.iloc[:1001,:], table_name="CustomerRaw", schema="Bronze", batch_size=1000)
    #conn.close()  # Close connection after insertion

✅ Step 1 Complete: DataFrame prepared in 0.01 sec
✅ Step 2 Complete: SQL Query Generated in 0.01 sec
✅ Step 3 Complete: DataFrame converted to tuples in 0.03 sec
✅ Inserted 1000/1001 rows in 92.31 sec
✅ Inserted 1001/1001 rows in 92.50 sec
✅ Successfully inserted 1001 records in 92.50 sec!
